![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark=SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [3]:
df = spark.read.format('csv')\
.option("delimiter", "\t")\
.option("inferSchema", "true")\
.load("SMSSpamCollection")


df.show(truncate=False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0 |_c1                                                                                                                                                                                                 |
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |
|ham |Ok lar... Joking wif u oni...                                                                                                                                                     

### Print the schema

In [4]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [5]:
df = df.withColumnRenamed("_c0","class") \
.withColumnRenamed("_c1","text")

In [6]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [7]:
df.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

In [8]:
df.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [9]:
df=df.select('*',(F.length(F.col('text')).alias('length')))

### Show the new dataframe

In [10]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [30]:
df.select('class','length').groupBy('class').agg(F.avg('length').alias('Avg. Lenght')).show()

+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [12]:
from pyspark.ml.feature import  IDF, Tokenizer,StopWordsRemover,CountVectorizer

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol='words',outputCol='newWords')
vectorizer = CountVectorizer(inputCol='newWords',outputCol='vectorized')
idf = IDF(inputCol="vectorized", outputCol="tfidf")


- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [14]:
from pyspark.ml.feature import StringIndexer,VectorAssembler


In [15]:
stringIndexer= StringIndexer(inputCol='class',outputCol='class_index',handleInvalid='skip')
vecAssembler= VectorAssembler(inputCols=['tfidf','length'],outputCol='features')


## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [16]:
from pyspark.ml.classification import NaiveBayes


In [17]:
nb=NaiveBayes(featuresCol='features',labelCol='class_index')


## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [18]:
from pyspark.ml.pipeline import Pipeline


In [19]:
pipline=Pipeline(stages=[tokenizer,remover,vectorizer,idf,stringIndexer,vecAssembler,nb])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [20]:
x_train, X_test = df.randomSplit([.7,.3],seed=42)

### Fit your Pipeline model to the training data

In [21]:
piplineModel=pipline.fit(x_train)

### Perform predictions on tests dataframe

In [22]:
predDF = piplineModel.transform(X_test)

### Print the schema of the prediction dataframe

In [23]:
predDF.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- newWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectorized: vector (nullable = true)
 |-- tfidf: vector (nullable = true)
 |-- class_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [25]:
evaluatore=MulticlassClassificationEvaluator(labelCol='class_index',predictionCol='prediction',metricName='f1')
acc=evaluatore.evaluate(predDF)


In [26]:
print('f1_score is:',acc)

f1_score is: 0.9727502290227267


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)